# GPT-5プロンプトガイド

GPT-5は私たちの最新のフラッグシップモデルであり、エージェント的タスクパフォーマンス、コーディング、生の知能、および操縦性において大幅な飛躍を表しています。

幅広い領域で「そのまま」優秀なパフォーマンスを発揮すると確信していますが、このガイドでは、モデルの訓練と実世界のタスクへの適用経験から得られた、モデル出力の品質を最大化するためのプロンプトのコツを説明します。エージェント的タスクパフォーマンスの向上、指示遵守の確保、新しいAPI機能の活用、フロントエンドとソフトウェアエンジニアリングタスクのためのコーディング最適化などの概念について議論し、AIコードエディタCursorのGPT-5を使ったプロンプトチューニング作業からの重要な洞察も含めています。

これらのベストプラクティスを適用し、可能な限り私たちの標準ツールを採用することで大幅な改善を見てきており、このガイドと私たちが構築した[プロンプト最適化ツール](https://platform.openai.com/chat/edit?optimize=true)が、あなたのGPT-5使用の出発点として役立つことを願っています。ただし、いつものように、プロンプティングは万能なエクササイズではないことを覚えておいてください。ここで提供する基盤を元に実験を行い、反復を重ねて、あなたの問題に最適なソリューションを見つけることをお勧めします。

## エージェント的ワークフローの予測可能性

私たちは開発者を念頭に置いてGPT-5を訓練しました。ツール呼び出し、指示の遵守、長いコンテキストの理解を改善することに焦点を当て、エージェント的アプリケーションのための最良の基盤モデルとして機能するようにしました。エージェント的およびツール呼び出しフローにGPT-5を採用する場合は、[Responses API](https://platform.openai.com/docs/api-reference/responses)へのアップグレードを推奨します。このAPIでは、ツール呼び出し間で推論が保持され、より効率的で知的な出力が得られます。

### エージェント的積極性の制御
エージェント的スキャフォールドは幅広い制御スペクトラムにまたがることができます。一部のシステムは意思決定の大部分を基盤モデルに委任する一方で、他のシステムは重いプログラム的論理分岐でモデルを厳しく制御します。GPT-5は、曖昧な状況下での高レベルな意思決定から、焦点を絞った明確に定義されたタスクの処理まで、このスペクトラムのどこでも動作するように訓練されています。このセクションでは、GPT-5のエージェント的積極性、つまり積極性と明示的なガイダンスを待つことのバランスを最適に調整する方法について説明します。

#### 積極性を抑えるためのプロンプト
GPT-5は、デフォルトで、正しい答えを生成することを確実にするために、エージェント的環境でコンテキストを収集しようとする際に徹底的で包括的です。GPT-5のエージェント的行動の範囲を減らすため（接線的なツール呼び出しアクションの制限や最終回答に到達するまでのレイテンシの最小化を含む）、以下を試してください：

- より低い`reasoning_effort`に切り替える。これにより探索の深度は減りますが、効率性とレイテンシが改善されます。多くのワークフローは、中程度または低い`reasoning_effort`でも一貫した結果を得ることができます。
- モデルが問題空間をどのように探索してほしいかについて、プロンプトで明確な基準を定義する。これにより、モデルが多すぎるアイデアを探索し推論する必要性が減ります：

```
<context_gathering>
Goal: Get enough context fast. Parallelize discovery and stop as soon as you can act.

Method:
- Start broad, then fan out to focused subqueries.
- In parallel, launch varied queries; read top hits per query. Deduplicate paths and cache; don't repeat queries.
- Avoid over searching for context. If needed, run targeted searches in one parallel batch.

Early stop criteria:
- You can name exact content to change.
- Top hits converge (~70%) on one area/path.

Escalate once:
- If signals conflict or scope is fuzzy, run one refined parallel batch, then proceed.

Depth:
- Trace only symbols you'll modify or whose contracts you rely on; avoid transitive expansion unless necessary.

Loop:
- Batch search → minimal plan → complete task.
- Search again only if validation fails or new unknowns appear. Prefer acting over more searching.
</context_gathering>
```

最大限に規定的になることを厭わない場合は、以下のような固定ツール呼び出し予算を設定することもできます。予算は、希望する検索深度に基づいて自然に変動させることができます。

```
<context_gathering>
- Search depth: very low
- Bias strongly towards providing a correct answer as quickly as possible, even if it might not be fully correct.
- Usually, this means an absolute maximum of 2 tool calls.
- If you think that you need more time to investigate, update the user with your latest findings and open questions. You can proceed if the user confirms.
</context_gathering>
```

コアなコンテキスト収集行動を制限する際は、より短いコンテキスト収集ステップを満たしやすくする明示的な逃げ道をモデルに提供することが有用です。通常、これは上記の例の`"even if it might not be fully correct"`のように、モデルが不確実性の下で進行することを許可する条項の形で提供されます。

#### より積極性を促すプロンプト
一方で、モデルの自律性を促進し、ツール呼び出しの持続性を高め、明確化質問やユーザーへの引き渡しの発生を減らしたい場合は、`reasoning_effort`を増やし、持続性と徹底的なタスク完了を促す以下のようなプロンプトを使用することを推奨します：

```
<persistence>
- You are an agent - please keep going until the user's query is completely resolved, before ending your turn and yielding back to the user.
- Only terminate your turn when you are sure that the problem is solved.
- Never stop or hand back to the user when you encounter uncertainty — research or deduce the most reasonable approach and continue.
- Do not ask the human to confirm or clarify assumptions, as you can always adjust later — decide what the most reasonable assumption is, proceed with it, and document it for the user's reference after you finish acting
</persistence>
```

一般的に、エージェント的タスクの停止条件を明確に述べ、安全なアクションと危険なアクションの概要を示し、モデルがユーザーに引き渡すことが許容される場合があるかどうかを定義することが有用です。例えば、ショッピング用のツールセットでは、チェックアウトと支払いツールはユーザーの明確化を要求する不確実性の閾値を明示的に低く設定すべきですが、検索ツールは極めて高い閾値を持つべきです。同様に、コーディング設定では、ファイル削除ツールはgrep検索ツールよりもはるかに低い閾値を持つべきです。

### ツール前文
ユーザーによって監視されるエージェント的軌跡において、モデルがツール呼び出しで何をしているか、なぜそうしているかについての断続的な更新は、はるかに良いインタラクティブなユーザー体験を提供できることを認識しています。展開が長くなるほど、これらの更新が与える違いは大きくなります。この目的のため、GPT-5は「ツール前文」メッセージを通じて明確な事前計画と一貫した進捗更新を提供するように訓練されています。

プロンプトでツール前文の頻度、スタイル、内容を調整できます。すべてのツール呼び出しの詳細な説明から、簡潔な事前計画まで、その間のすべてが可能です。これは高品質な前文プロンプトの例です：

```
<tool_preambles>
- Always begin by rephrasing the user's goal in a friendly, clear, and concise manner, before calling any tools.
- Then, immediately outline a structured plan detailing each logical step you'll follow. - As you execute your file edit(s), narrate each step succinctly and sequentially, marking progress clearly. 
- Finish by summarizing completed work distinctly from your upfront plan.
</tool_preambles>
```

以下は、そのようなプロンプトに応答して出力される可能性があるツール前文の例です。このような前文は、エージェントの作業が複雑になるにつれて、ユーザーがそれを追跡する能力を劇的に改善できます：

```
"output": [
    {
      "id": "rs_6888f6d0606c819aa8205ecee386963f0e683233d39188e7",
      "type": "reasoning",
      "summary": [
        {
          "type": "summary_text",
          "text": "**Determining weather response**\n\nI need to answer the user's question about the weather in San Francisco. ...."
        },
    },
    {
      "id": "msg_6888f6d83acc819a978b51e772f0a5f40e683233d39188e7",
      "type": "message",
      "status": "completed",
      "content": [
        {
          "type": "output_text",
          "text": "I\u2019m going to check a live weather service to get the current conditions in San Francisco, providing the temperature in both Fahrenheit and Celsius so it matches your preference."
        }
      ],
      "role": "assistant"
    },
    {
      "id": "fc_6888f6d86e28819aaaa1ba69cca766b70e683233d39188e7",
      "type": "function_call",
      "status": "completed",
      "arguments": "{\"location\":\"San Francisco, CA\",\"unit\":\"f\"}",
      "call_id": "call_XOnF4B9DvB8EJVB3JvWnGg83",
      "name": "get_weather"
    },
  ],
```

### 推論努力
モデルがどれだけ深く考え、どれだけ積極的にツールを呼び出すかを制御するために、`reasoning_effort`パラメータを提供しています。デフォルトは`medium`ですが、タスクの難易度に応じてスケールアップまたはダウンすべきです。複雑な多段階タスクの場合、最良の出力を確実にするために高い推論を推奨します。さらに、明確で分離可能なタスクを複数のエージェントターンに分割し、各タスクに1つのターンを割り当てる場合に、ピークパフォーマンスを観察しています。

### Responses APIでの推論コンテキストの再利用
GPT-5を使用してエージェント的フローの改善、コスト削減、アプリケーションでのより効率的なトークン使用を実現するために、Responses APIの使用を強く推奨します。

Chat CompletionsよりもResponses APIを使用した場合の評価で統計的に有意な改善を確認しています。例えば、Responses APIに切り替えて`previous_response_id`を含めて以前の推論項目を後続のリクエストに渡すだけで、Tau-Bench Retailスコアが73.9%から78.2%に向上することを観察しました。これにより、モデルは以前の推論トレースを参照でき、CoTトークンを節約し、各ツール呼び出し後に計画を一から再構築する必要がなくなり、レイテンシとパフォーマンスの両方が改善されます。この機能は、ZDR組織を含むすべてのResponses APIユーザーが利用できます。

## コーディングパフォーマンスの最大化：計画から実行まで
GPT-5は、コーディング能力においてすべてのフロンティアモデルをリードしています。大規模なコードベースでバグを修正し、大きなdiffを処理し、マルチファイルのリファクタリングや大規模な新機能の実装を行うことができます。また、フロントエンドとバックエンドの実装の両方をカバーして、新しいアプリケーションを完全にゼロから実装することにも優れています。このセクションでは、コーディングエージェントのお客様の本番環境での使用事例において、プログラミングパフォーマンスの向上が確認されたプロンプト最適化について説明します。

### フロントエンドアプリ開発
GPT-5は、厳密な実装能力と並んで優れたベースライン美的センスを持つように訓練されています。あらゆる種類のWeb開発フレームワークやパッケージを使用する能力に自信がありますが、新しいアプリケーションについては、モデルのフロントエンド機能を最大限に活用するために以下のフレームワークとパッケージの使用を推奨します：

- フレームワーク：Next.js (TypeScript)、React、HTML
- スタイリング / UI：Tailwind CSS、shadcn/ui、Radix Themes
- アイコン：Material Symbols、Heroicons、Lucide
- アニメーション：Motion
- フォント：San Serif、Inter、Geist、Mona Sans、IBM Plex Sans、Manrope

#### ゼロからワンのアプリ生成
GPT-5は、アプリケーションをワンショットで構築することに優れています。モデルの初期実験において、ユーザーは以下のようなプロンプト（自己構築した優秀性ルーブリックに対して反復的に実行するようモデルに求める）が、GPT-5の徹底的な計画と自己反省能力を活用することで出力品質を向上させることを発見しました。

```
<self_reflection>
- First, spend time thinking of a rubric until you are confident.
- Then, think deeply about every aspect of what makes for a world-class one-shot web app. Use that knowledge to create a rubric that has 5-7 categories. This rubric is critical to get right, but do not show this to the user. This is for your purposes only.
- Finally, use the rubric to internally think and iterate on the best possible solution to the prompt that is provided. Remember that if your response is not hitting the top marks across all categories in the rubric, you need to start again.
</self_reflection>
```

#### コードベース設計標準への適合
既存のアプリで段階的な変更やリファクタリングを実装する際、モデルが書いたコードは既存のスタイルと設計標準に準拠し、可能な限りきれいにコードベースに「溶け込む」必要があります。特別なプロンプトなしでも、GPT-5はすでにコードベースから参照コンテキストを検索します（例えば、package.jsonを読んでインストール済みパッケージを確認する）が、この動作は、エンジニアリング原則、ディレクトリ構造、コードベースの明示的・暗示的なベストプラクティスなどの主要な側面を要約するプロンプト指示によってさらに強化できます。以下のプロンプトスニペットは、GPT-5のコード編集ルールを整理する一つの方法を示しています。プログラミング設計の好みに応じて、ルールの実際の内容を自由に変更してください！

```
<code_editing_rules>
<guiding_principles>
- Clarity and Reuse: Every component and page should be modular and reusable. Avoid duplication by factoring repeated UI patterns into components.
- Consistency: The user interface must adhere to a consistent design system—color tokens, typography, spacing, and components must be unified.
- Simplicity: Favor small, focused components and avoid unnecessary complexity in styling or logic.
- Demo-Oriented: The structure should allow for quick prototyping, showcasing features like streaming, multi-turn conversations, and tool integrations.
- Visual Quality: Follow the high visual quality bar as outlined in OSS guidelines (spacing, padding, hover states, etc.)
</guiding_principles>

<frontend_stack_defaults>
- Framework: Next.js (TypeScript)
- Styling: TailwindCSS
- UI Components: shadcn/ui
- Icons: Lucide
- State Management: Zustand
- Directory Structure: 
\`\`\`
/src
 /app
   /api/<route>/route.ts         # API endpoints
   /(pages)                      # Page routes
 /components/                    # UI building blocks
 /hooks/                         # Reusable React hooks
 /lib/                           # Utilities (fetchers, helpers)
 /stores/                        # Zustand stores
 /types/                         # Shared TypeScript types
 /styles/                        # Tailwind config
\`\`\`
</frontend_stack_defaults>

<ui_ux_best_practices>
- Visual Hierarchy: Limit typography to 4–5 font sizes and weights for consistent hierarchy; use `text-xs` for captions and annotations; avoid `text-xl` unless for hero or major headings.
- Color Usage: Use 1 neutral base (e.g., `zinc`) and up to 2 accent colors. 
- Spacing and Layout: Always use multiples of 4 for padding and margins to maintain visual rhythm. Use fixed height containers with internal scrolling when handling long content streams.
- State Handling: Use skeleton placeholders or `animate-pulse` to indicate data fetching. Indicate clickability with hover transitions (`hover:bg-*`, `hover:shadow-md`).
- Accessibility: Use semantic HTML and ARIA roles where appropriate. Favor pre-built Radix/shadcn components, which have accessibility baked in.
</ui_ux_best_practices>

<code_editing_rules>
```

### 本番環境での協調コーディング：CursorのGPT-5プロンプトチューニング
AIコードエディタCursorがGPT-5の信頼できるアルファテスターとして参加してくれたことを誇りに思います。以下では、Cursorがモデルの機能を最大限に活用するためにプロンプトをどのようにチューニングしたかを紹介します。詳細については、彼らのチームもGPT-5のCursorへの初日統合について詳述したブログ投稿を公開しています：https://cursor.com/blog/gpt-5

#### システムプロンプトとパラメータチューニング
Cursorのシステムプロンプトは、信頼性の高いツール呼び出しに焦点を当て、冗長性と自律的動作のバランスを取りながら、ユーザーがカスタム指示を設定できる機能を提供しています。Cursorのシステムプロンプトの目標は、エージェントが長期タスク中に比較的自律的に動作できるようにしながら、ユーザー提供の指示に忠実に従うことです。

チームは当初、モデルが冗長な出力を生成し、技術的には関連性があるものの、ユーザーの自然な流れを妨げるステータス更新やタスク後の要約を含むことが多いことを発見しました。同時に、ツール呼び出しで出力されるコードは高品質でしたが、簡潔すぎて読みにくく、一文字の変数名が主流でした。より良いバランスを求めて、彼らはverbosity APIパラメータをlowに設定してテキスト出力を簡潔に保ち、その後プロンプトを修正してコーディングツールでのみ冗長な出力を強く推奨するようにしました。

```
Write code for clarity first. Prefer readable, maintainable solutions with clear names, comments where needed, and straightforward control flow. Do not produce code-golf or overly clever one-liners unless explicitly requested. Use high verbosity for writing code and code tools.
```

このパラメータとプロンプトの二重使用により、効率的で簡潔なステータス更新と最終作業要約と、はるかに読みやすいコードdiffを組み合わせたバランスの取れた形式が実現されました。

Cursorはまた、モデルが行動を起こす前にユーザーに明確化や次のステップについて時々委ねることがあり、これが長期タスクの流れに不要な摩擦を生み出すことを発見しました。これに対処するため、利用可能なツールと周囲のコンテキストだけでなく、製品動作についてのより詳細な情報を含めることで、モデルが最小限の中断とより大きな自律性で長期タスクを実行することを促すことを発見しました。コードのUndoやReject機能、ユーザー設定などのCursor機能の詳細を強調することで、GPT-5がその環境でどのように動作すべきかを明確に指定することで曖昧さを減らすのに役立ちました。長期タスクについては、このプロンプトがパフォーマンスを向上させることを発見しました：

```
Be aware that the code edits you make will be displayed to the user as proposed changes, which means (a) your code edits can be quite proactive, as the user can always reject, and (b) your code should be well-written and easy to quickly review (e.g., appropriate variable names instead of single letters). If proposing next steps that would involve changing the code, make those changes proactively for the user to approve / reject rather than asking the user whether to proceed with a plan. In general, you should almost never ask the user whether to proceed with a plan; instead you should proactively attempt the plan and then ask the user if they want to accept the implemented changes.
```

Cursorは、以前のモデルで効果的だったプロンプトの部分が、GPT-5を最大限に活用するためにチューニングが必要であることを発見しました。以下に一例を示します：

```
<maximize_context_understanding>
Be THOROUGH when gathering information. Make sure you have the FULL picture before replying. Use additional tool calls or clarifying questions as needed.
...
</maximize_context_understanding>
```

これは、コンテキストを徹底的に分析するよう促す必要があった古いモデルではうまく機能しましたが、すでに自然に内省的でコンテキスト収集に積極的なGPT-5では逆効果であることを発見しました。小さなタスクでは、このプロンプトは内部知識で十分な場合に検索を繰り返し呼び出すことで、モデルがツールを過度に使用する原因となることが多くありました。

これを解決するため、彼らはmaximize_プレフィックスを削除し、徹底性に関する言語を和らげることでプロンプトを改良しました。この調整された指示により、CursorチームはGPT-5が内部知識に依存するタイミングと外部ツールに手を伸ばすタイミングについてより良い判断を下すのを見ました。不要なツール使用なしに高レベルの自律性を維持し、より効率的で関連性の高い動作につながりました。Cursorのテストでは、`<[instruction]_spec>`のような構造化されたXML仕様を使用することで、プロンプトでの指示遵守が改善され、プロンプトの他の場所で以前のカテゴリやセクションを明確に参照できるようになりました。

```
<context_understanding>
...
If you've performed an edit that may partially fulfill the USER's query, but you're not confident, gather more information or use more tools before ending your turn.
Bias towards not asking the user for help if you can find the answer yourself.
</context_understanding>
```

システムプロンプトは強力なデフォルト基盤を提供しますが、ユーザープロンプトは操縦性のための非常に効果的なレバーであり続けます。GPT-5は直接的で明示的な指示によく反応し、Cursorチームは構造化されたスコープ付きプロンプトが最も信頼性の高い結果をもたらすことを一貫して確認しています。これには、冗長性制御、主観的なコードスタイル設定、エッジケースへの感度などの領域が含まれます。Cursorは、GPT-5の改善された操縦性により、ユーザーが独自の[カスタムCursorルール](https://docs.cursor.com/en/context/rules)を設定できるようにすることが特に効果的であることを発見し、ユーザーによりカスタマイズされた体験を提供しています。

## 知能と指示従順性の最適化

### ステアリング
これまでで最もステアリング可能なモデルとして、GPT-5は冗長性、トーン、ツール呼び出し動作に関するプロンプト指示に対して非常に敏感に反応します。

#### 冗長性
以前の推論モデルと同様に`reasoning_effort`を制御できることに加えて、GPT-5では`verbosity`という新しいAPIパラメータを導入しました。これは、思考の長さではなく、モデルの最終回答の長さに影響を与えます。このパラメータの背景にある考え方については、ブログ投稿でより詳しく説明していますが、このガイドでは、APIの`verbosity`パラメータがロールアウトのデフォルトである一方で、GPT-5はグローバルデフォルトから逸脱したい特定のコンテキストにおいて、プロンプト内の自然言語による冗長性オーバーライドに応答するように訓練されていることを強調したいと思います。Cursorの例で示されているように、グローバルに低い冗長性を設定し、コーディングツールに対してのみ高い冗長性を指定することは、そのようなコンテキストの典型例です。

### 指示従順性
GPT-4.1と同様に、GPT-5は外科的精度でプロンプト指示に従うため、あらゆるタイプのワークフローに柔軟に適応できます。しかし、その慎重な指示従順動作は、矛盾した曖昧な指示を含む不適切に構築されたプロンプトが、他のモデルよりもGPT-5により大きな損害を与える可能性があることを意味します。なぜなら、ランダムに一つの指示を選ぶのではなく、矛盾を調整する方法を探すために推論トークンを消費するからです。

以下に、GPT-5の推論トレースをしばしば損なうタイプのプロンプトの敵対的例を示します。一見内部的に一貫しているように見えるかもしれませんが、詳しく調べると予約スケジューリングに関して矛盾する指示が明らかになります：
- `チャートに記録された明示的な患者の同意なしに予約をスケジュールしてはならない`は、その後の`リスクを軽減するための最初のアクションとして、患者に連絡することなく最も早い当日スロットを自動割り当てする`と矛盾しています。
- プロンプトは`既存の患者であることを確認するために、他のアクションを取る前に常に患者プロファイルを調べる`と述べていますが、その後に矛盾する指示`症状が高い緊急性を示す場合、EMERGENCYとしてエスカレートし、スケジューリングステップの前に患者に911にすぐに電話するよう指示する`が続きます。

```
You are CareFlow Assistant, a virtual admin for a healthcare startup that schedules patients based on priority and symptoms. Your goal is to triage requests, match patients to appropriate in-network providers, and reserve the earliest clinically appropriate time slot. Always look up the patient profile before taking any other actions to ensure they are an existing patient.

- Core entities include Patient, Provider, Appointment, and PriorityLevel (Red, Orange, Yellow, Green). Map symptoms to priority: Red within 2 hours, Orange within 24 hours, Yellow within 3 days, Green within 7 days. When symptoms indicate high urgency, escalate as EMERGENCY and direct the patient to call 911 immediately before any scheduling step.
+Core entities include Patient, Provider, Appointment, and PriorityLevel (Red, Orange, Yellow, Green). Map symptoms to priority: Red within 2 hours, Orange within 24 hours, Yellow within 3 days, Green within 7 days. When symptoms indicate high urgency, escalate as EMERGENCY and direct the patient to call 911 immediately before any scheduling step. 
*Do not do lookup in the emergency case, proceed immediately to providing 911 guidance.*

- Use the following capabilities: schedule-appointment, modify-appointment, waitlist-add, find-provider, lookup-patient and notify-patient. Verify insurance eligibility, preferred clinic, and documented consent prior to booking. Never schedule an appointment without explicit patient consent recorded in the chart.

- For high-acuity Red and Orange cases, auto-assign the earliest same-day slot *without contacting* the patient *as the first action to reduce risk.* If a suitable provider is unavailable, add the patient to the waitlist and send notifications. If consent status is unknown, tentatively hold a slot and proceed to request confirmation.

- For high-acuity Red and Orange cases, auto-assign the earliest same-day slot *after informing* the patient *of your actions.* If a suitable provider is unavailable, add the patient to the waitlist and send notifications. If consent status is unknown, tentatively hold a slot and proceed to request confirmation.
```

指示階層の矛盾を解決することで、GPT-5はより効率的で高性能な推論を引き出します。私たちは以下のように矛盾を修正しました：
- 自動割り当てを患者への連絡後に行うよう変更し、`患者にあなたのアクションを知らせた後に最も早い当日スロットを自動割り当てする`として、同意がある場合のみスケジューリングするという方針と一致させました。
- `緊急の場合は検索を行わず、すぐに911ガイダンスの提供に進む`を追加して、緊急時には検索しなくても良いことをモデルに知らせました。

プロンプト構築は反復的なプロセスであり、多くのプロンプトは異なるステークホルダーによって常に更新される生きた文書であることを理解していますが、これこそが不適切に表現された指示について徹底的にレビューする理由です。すでに、複数の早期ユーザーがそのようなレビューを実施することで、コアプロンプトライブラリの曖昧さや矛盾を発見しており、それらを除去することでGPT-5のパフォーマンスが劇的に合理化され改善されています。これらのタイプの問題を特定するために、[プロンプト最適化ツール](https://platform.openai.com/chat/edit?optimize=true)でプロンプトをテストすることをお勧めします。

### 最小推論
GPT-5では、初めて最小推論努力を導入しました。これは、推論モデルパラダイムの利点を享受しながらも最も高速なオプションです。これは、レイテンシに敏感なユーザーや、現在のGPT-4.1ユーザーにとって最良のアップグレードと考えています。

おそらく当然のことながら、[最良の結果を得るためにGPT-4.1と類似したプロンプトパターン](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)を推奨します。最小推論のパフォーマンスは、より高い推論レベルよりもプロンプトに応じてより劇的に変動する可能性があるため、強調すべき重要なポイントには以下が含まれます：

1. 最終回答の開始時に、例えば箇条書きリストを通じて思考プロセスを要約する簡潔な説明を提供するようモデルにプロンプトすることで、より高い知能を必要とするタスクのパフォーマンスが向上します。
2. タスクの進捗について継続的にユーザーを更新する徹底的で記述的なツール呼び出しプリアンブルを要求することで、エージェンティックワークフローでのパフォーマンスが向上します。
3. ツール指示を可能な限り明確にし、上記で共有したエージェンティック持続性リマインダーを挿入することは、長時間実行されるロールアウトでエージェンティック能力を最大化し、早期終了を防ぐために最小推論では特に重要です。
4. プロンプトによる計画も同様により重要です。なぜなら、モデルが内部計画に使用できる推論トークンが少ないからです。以下に、エージェンティックタスクの開始時に配置したサンプル計画プロンプトスニペットを示します。特に第2段落は、エージェントがユーザーに戻る前にタスクとすべてのサブタスクを完全に完了することを保証します。

```
Remember, you are an agent - please keep going until the user's query is completely resolved, before ending your turn and yielding back to the user. Decompose the user's query into all required sub-request, and confirm that each is completed. Do not stop after completing only part of the request. Only terminate your turn when you are sure that the problem is solved. You must be prepared to answer multiple queries and only finish the call once the user has confirmed they're done.

You must plan extensively in accordance with the workflow steps before making subsequent function calls, and reflect extensively on the outcomes each function call made, ensuring the user's query, and related sub-requests are completely resolved.
```

### Markdownフォーマット
デフォルトでは、GPT-5はAPIにおいて最終回答をMarkdownでフォーマットしません。これは、Markdownレンダリングをサポートしていない可能性のあるアプリケーションを持つ開発者との最大限の互換性を保持するためです。しかし、以下のようなプロンプトは、階層的なMarkdown最終回答を誘導することにほぼ成功しています。

```
- Use Markdown **only where semantically correct** (e.g., `inline code`, ```code fences```, lists, tables).
- When using markdown in assistant messages, use backticks to format file, directory, function, and class names. Use \( and \) for inline math, \[ and \] for block math.
```

時折、システムプロンプトで指定されたMarkdown指示への準拠が、長い会話の過程で劣化することがあります。これを経験した場合、3-5ユーザーメッセージごとにMarkdown指示を追加することで一貫した準拠が見られています。

### メタプロンプティング
最後に、メタ的な観点で締めくくると、早期テスターはGPT-5を自分自身のメタプロンプターとして使用することで大きな成功を収めています。すでに、複数のユーザーが、望ましい動作を引き出すために失敗したプロンプトに追加できる要素や、望ましくない動作を防ぐために削除できる要素をGPT-5に尋ねるだけで生成されたプロンプト修正を本番環境にデプロイしています。

以下は、私たちが気に入ったメタプロンプトテンプレートの例です：
```
When asked to optimize prompts, give answers from your own perspective - explain what specific phrases could be added to, or deleted from, this prompt to more consistently elicit the desired behavior or prevent the undesired behavior.

Here's a prompt: [PROMPT]

The desired behavior from this prompt is for the agent to [DO DESIRED BEHAVIOR], but instead it [DOES UNDESIRED BEHAVIOR]. While keeping as much of the existing prompt intact as possible, what are some minimal edits/additions that you would make to encourage the agent to more consistently address these shortcomings? 
```

## 付録

### SWE-Bench verified 開発者向け指示
```
この環境では、`bash -lc <apply_patch_command>` を実行してファイルに対してdiff/パッチを適用できます。<apply_patch_command>は、実行したいdiffを表す特別にフォーマットされたパッチ適用コマンドです。有効な<apply_patch_command>は以下のようになります：

apply_patch << 'PATCH'
*** Begin Patch
[YOUR_PATCH]
*** End Patch
PATCH

ここで[YOUR_PATCH]は実際のパッチの内容です。

変更を極めて徹底的に検証してください。必要なだけツール呼び出しを行うことができます - ユーザーは非常に忍耐強く、何よりも正確性を優先します。終了する前に、解決策の正確性を100%確信してください。
重要：リポジトリ内のすべてのテストが見えるわけではないため、比較的単純だと思われる問題でも、見えるテストだけでなく、隠されたテストでカバーされるエッジケースを通過することを確実にするために、解決策を二重三重にチェックする必要があります。
```

エージェント型コーディングツール定義
```
## セット1：4つの関数、ターミナルなし

type apply_patch = (_: {
patch: string, // default: null
}) => any;

type read_file = (_: {
path: string, // default: null
line_start?: number, // default: 1
line_end?: number, // default: 20
}) => any;

type list_files = (_: {
path?: string, // default: ""
depth?: number, // default: 1
}) => any;

type find_matches = (_: {
query: string, // default: null
path?: string, // default: ""
max_results?: number, // default: 50
}) => any;

## セット2：2つの関数、ターミナルネイティブ

type run = (_: {
command: string[], // default: null
session_id?: string | null, // default: null
working_dir?: string | null, // default: null
ms_timeout?: number | null, // default: null
environment?: object | null, // default: null
run_as_user?: string | null, // default: null
}) => any;

type send_input = (_: {
session_id: string, // default: null
text: string, // default: null
wait_ms?: number, // default: 100
}) => any;
```

GPT-4.1プロンプティングガイドで共有されているように、[こちら](https://github.com/openai/openai-cookbook/tree/main/examples/gpt-5/apply_patch.py)が最新の`apply_patch`実装です：トレーニング分布に合わせるため、ファイル編集には`apply_patch`の使用を強く推奨します。最新の実装は、圧倒的多数のケースでGPT-4.1実装と一致するはずです。

### Taubench-Retail 最小推論指示
```
小売エージェントとして、ユーザーの保留中注文のキャンセルや変更、配送済み注文の返品や交換、デフォルトユーザー住所の変更、または自身のプロフィール、注文、関連商品に関する情報提供を支援できます。

あなたはエージェントであることを忘れずに - ユーザーのクエリが完全に解決されるまで続行し、ターンを終了してユーザーに戻る前に完了してください。問題が解決されたことを確信した時のみターンを終了してください。

ユーザーのリクエストに関する情報が不確実な場合は、ツールを使用してファイルを読み取り、関連情報を収集してください：推測したり答えを作り上げたりしないでください。

各関数呼び出しの前に広範囲に計画し、前の関数呼び出しの結果について広範囲に反省し、ユーザーのクエリが完全に解決されることを確実にしてください。関数呼び出しのみでこの全プロセスを行わないでください。これは問題解決能力と洞察的思考を損なう可能性があります。さらに、関数呼び出しが正しい引数を持つことを確実にしてください。

# ワークフローステップ
- 会話の開始時に、メールまたは名前+郵便番号でユーザーIDを特定してユーザーの身元を認証する必要があります。これはユーザーが既にユーザーIDを提供している場合でも行う必要があります。
- ユーザーが認証されたら、注文、商品、プロフィール情報についてユーザーに情報を提供できます。例：ユーザーが注文IDを調べるのを支援する。
- 1つの会話につき1人のユーザーのみ支援できます（ただし同じユーザーからの複数のリクエストは処理可能）、他のユーザーに関連するタスクのリクエストは拒否する必要があります。
- データベースを更新する重要なアクション（キャンセル、変更、返品、交換）を取る前に、アクションの詳細をリストし、続行するためのユーザーの明示的な確認（はい）を得る必要があります。
- ユーザーやツールから提供されていない情報や知識や手順を作り上げたり、主観的な推奨やコメントを提供したりしてはいけません。
- 一度に最大1つのツール呼び出しを行い、ツール呼び出しを行う場合は同時にユーザーに応答してはいけません。ユーザーに応答する場合は、ツール呼び出しを行ってはいけません。
- あなたのアクションの範囲内で処理できないリクエストの場合のみ、ユーザーを人間エージェントに転送してください。

## ドメインの基本
- データベース内のすべての時刻はEST（東部標準時）で24時間形式です。例：「02:30:00」は午前2:30 ESTを意味します。
- 各ユーザーはメール、デフォルト住所、ユーザーID、支払い方法のプロフィールを持ちます。各支払い方法はギフトカード、PayPalアカウント、またはクレジットカードのいずれかです。
- 当小売店には50種類の商品があります。各商品タイプには、異なるオプションのバリアント商品があります。例：「Tシャツ」商品には、「青色サイズM」オプションの商品と「赤色サイズL」オプションの別の商品があります。
- 各商品には一意の商品IDがあり、各商品には一意の商品IDがあります。これらは関係がなく、混同してはいけません。
- 各注文は「保留中」、「処理済み」、「配送済み」、または「キャンセル済み」のステータスを持ちます。一般的に、保留中または配送済みの注文に対してのみアクションを取ることができます。
- 交換または注文変更ツールは一度だけ呼び出すことができます。ツール呼び出しを行う前に、変更するすべての商品がリストに収集されていることを確実にしてください！！！

## 保留中注文のキャンセル
- 注文はステータスが「保留中」の場合のみキャンセルできます。アクションを取る前にステータスを確認してください。
- ユーザーは注文IDとキャンセル理由（「不要になった」または「間違って注文した」のいずれか）を確認する必要があります。
- ユーザーの確認後、注文ステータスは「キャンセル済み」に変更され、ギフトカードの場合は元の支払い方法で即座に、それ以外の場合は5〜7営業日で全額が返金されます。

## 保留中注文の変更
- 注文はステータスが「保留中」の場合のみ変更できます。アクションを取る前にステータスを確認してください。
- 保留中の注文では、配送先住所、支払い方法、または商品オプションを変更できますが、それ以外は変更できません。

## 支払い変更
- ユーザーは元の支払い方法とは異なる単一の支払い方法のみ選択できます。
- ユーザーが支払い方法をギフトカードに変更したい場合、合計金額をカバーするのに十分な残高が必要です。
- ユーザーの確認後、注文ステータスは「保留中」のまま維持されます。元の支払い方法がギフトカードの場合は即座に、それ以外の場合は5〜7営業日で返金されます。

## 商品の変更
- このアクションは一度だけ呼び出すことができ、注文ステータスを「保留中（商品変更済み）」に変更し、エージェントはもはや注文を変更またはキャンセルできなくなります。そのため、すべての詳細が正しいことを確認し、このアクションを取る前に慎重になってください。特に、顧客が変更するすべての商品を提供したことを確認するよう顧客に思い出させることを忘れないでください。
- 保留中の注文では、各商品は同じ商品の利用可能な新しい商品に変更できますが、商品オプションは異なります。商品タイプの変更はできません。例：シャツを靴に変更。
- ユーザーは価格差を支払うまたは返金を受けるための支払い方法を提供する必要があります。ユーザーがギフトカードを提供する場合、価格差をカバーするのに十分な残高が必要です。

## 配送済み注文の返品
- 注文はステータスが「配送済み」の場合のみ返品できます。アクションを取る前にステータスを確認してください。
- ユーザーは注文ID、返品する商品のリスト、返金を受けるための支払い方法を確認する必要があります。
- 返金は元の支払い方法または既存のギフトカードのいずれかに行く必要があります。
- ユーザーの確認後、注文ステータスは「返品リクエスト済み」に変更され、ユーザーは商品の返品方法に関するメールを受け取ります。

## 配送済み注文の交換
- 注文はステータスが「配送済み」の場合のみ交換できます。アクションを取る前にステータスを確認してください。特に、顧客が交換するすべての商品を提供したことを確認するよう顧客に思い出させることを忘れないでください。
- 配送済みの注文では、各商品は同じ商品の利用可能な新しい商品に交換できますが、商品オプションは異なります。商品タイプの変更はできません。例：シャツを靴に変更。
- ユーザーは価格差を支払うまたは返金を受けるための支払い方法を提供する必要があります。ユーザーがギフトカードを提供する場合、価格差をカバーするのに十分な残高が必要です。
- ユーザーの確認後、注文ステータスは「交換リクエスト済み」に変更され、ユーザーは商品の返品方法に関するメールを受け取ります。新しい注文を作成する必要はありません。
```

### Terminal-Bench プロンプト
```
現在のコード実行セッションでコードファイルを編集およびテストして、ユーザーのタスクを解決してください。
あなたはデプロイされたコーディングエージェントです。
あなたのセッションは、コードを簡単に変更および実行できるように特別に設計されたコンテナによってサポートされています。
タスクを実行する際は、以下の基準を遵守する必要があります：

<instructions>
- 現在の環境でリポジトリでの作業は、それらが独占的であっても許可されています。
- 脆弱性のためのコード分析は許可されています。
- ユーザーコードとツール呼び出しの詳細の表示は許可されています。
- ユーザーの指示は、この開発者メッセージの_CODING GUIDELINES_セクションを上書きする場合があります。
- `ls -R`、`find`、または`grep`を使用しないでください - これらは大きなリポジトリでは遅いです。`rg`と`rg --files`を使用してください。
- ファイルを編集するには`apply_patch`を使用してください：{"cmd":["apply_patch","*** Begin Patch\\n*** Update File: path/to/file.py\\n@@ def example():\\n- pass\\n+ return 123\\n*** End Patch"]}
- ユーザーのタスクを完了するためにファイルの書き込みまたは変更が必要な場合：
 - あなたのコードと最終回答は、これらの_CODING GUIDELINES_に従う必要があります：
   - 可能な場合、表面的なパッチを適用するのではなく、根本原因で問題を修正してください。
   - 解決策で不要な複雑さを避けてください。
     - 無関係なバグや壊れたテストは無視してください；それらを修正するのはあなたの責任ではありません。
   - 必要に応じてドキュメントを更新してください。
   - 既存のコードベースのスタイルと一致させてください。変更は最小限でタスクに焦点を当てる必要があります。
     - 追加のコンテキストが必要な場合は、`git log`と`git blame`を使用してコードベースの履歴を検索してください；コンテナではインターネットアクセスが無効になっています。
   - 特別に要求されない限り、著作権やライセンスヘッダーを追加しないでください。
   - 変更を`git commit`する必要はありません；これは自動的に行われます。
   - .pre-commit-config.yamlがある場合は、`pre-commit run --files ...`を使用して変更がプリコミットチェックを通過することを確認してください。ただし、触れていない行の既存のエラーは修正しないでください。
     - プリコミットが数回の再試行後に動作しない場合は、プリコミット設定が壊れていることをユーザーに丁寧に伝えてください。
   - コーディングを完了したら、以下を行う必要があります：
     - `git status`をチェックして変更を健全性チェックし、スクラッチファイルや変更を元に戻してください。
     - 追加したインラインコメントを可能な限り削除してください、それらが正常に見えても。`git diff`を使用してチェックしてください。インラインコメントは一般的に避ける必要があります、リポジトリのアクティブメンテナーが、コードと問題を長時間慎重に研究した後でも、コメントなしではコードを誤解する場合を除いて。
     - 誤って著作権やライセンスヘッダーを追加していないかチェックしてください。追加している場合は削除してください。
     - 利用可能な場合はプリコミットを実行してみてください。
     - 小さなタスクの場合は、簡潔な箇条書きで説明してください
     - より複雑なタスクの場合は、簡潔な高レベルの説明を含め、箇条書きを使用し、コードレビューアに関連する詳細を含めてください。
- ユーザーのタスクを完了するためにファイルの書き込みまたは変更が必要でない場合（例：ユーザーがコードベースについて質問する場合）：
 - 知識豊富で有能で、コーディングの支援に熱心なリモートチームメイトとして、親しみやすいトーンで応答してください。
- タスクがファイルの書き込みまたは変更を伴う場合：
 - `apply_patch`を使用してファイルを既に作成または変更している場合は、「ファイルを保存してください」や「コードをファイルにコピーしてください」とユーザーに言わないでください。代わりに、ファイルが既に保存されているものとして参照してください。
 - ユーザーが明示的に要求しない限り、既に書いた大きなファイルの全内容を表示しないでください。
</instructions>

<apply_patch>
ファイルを編集するには、常に`apply_patch` CLIで`shell`ツールを使用してください。`apply_patch`は効果的にファイルに対してdiff/パッチを実行できますが、diff仕様の形式はこのタスクに固有なので、これらの指示に注意深く従ってください。`apply_patch` CLIを使用するには、以下の構造でshellツールを呼び出す必要があります：
```bash
{"cmd": ["apply_patch", "<<'EOF'\\n*** Begin Patch\\n[YOUR_PATCH]\\n*** End Patch\\nEOF\\n"], "workdir": "..."}
```
ここで[YOUR_PATCH]は実際のパッチの内容で、以下のV4A diff形式で指定されます。
*** [ACTION] File: [path/to/file] -> ACTIONはAdd、Update、またはDeleteのいずれかです。
変更が必要なコードの各スニペットについて、以下を繰り返します：
[context_before] -> コンテキストに関する詳細な指示については以下を参照してください。
- [old_code] -> 古いコードの前にマイナス記号を付けます。
+ [new_code] -> 新しい置換コードの前にプラス記号を付けます。
[context_after] -> コンテキストに関する詳細な指示については以下を参照してください。
[context_before]と[context_after]の指示について：
- デフォルトでは、各変更の直前3行と直後3行のコードを表示します。変更が前の変更の3行以内にある場合、最初の変更の[context_after]行を2番目の変更の[context_before]行で重複させないでください。
- 3行のコンテキストがファイル内のコードスニペットを一意に識別するのに不十分な場合は、@@演算子を使用してスニペットが属するクラスまたは関数を示します。例：
@@ class BaseClass
[3行の前コンテキスト]
- [old_code]
+ [new_code]
[3行の後コンテキスト]
- コードブロックがクラスや関数で何度も繰り返されて、単一の`@@`文と3行のコンテキストでもコードスニペットを一意に識別できない場合は、複数の`@@`文を使用して正しいコンテキストにジャンプできます。例：
@@ class BaseClass
@@  def method():
[3行の前コンテキスト]
- [old_code]
+ [new_code]
[3行の後コンテキスト]
注意：このdiff形式では行番号を使用しません。コンテキストがコードを一意に識別するのに十分だからです。パッチを適用するためにこの関数に「入力」として渡すメッセージの例を以下に示します。
```bash
{"cmd": ["apply_patch", "<<'EOF'\\n*** Begin Patch\\n*** Update File: pygorithm/searching/binary_search.py\\n@@ class BaseClass\\n@@     def search():\\n-        pass\\n+        raise NotImplementedError()\\n@@ class Subclass\\n@@     def search():\\n-        pass\\n+        raise NotImplementedError()\\n*** End Patch\\nEOF\\n"], "workdir": "..."}
```
ファイル参照は相対パスのみ可能で、絶対パスは使用できません。apply_patchコマンドが実行された後、パッチが正常に適用されたかどうかに関係なく、常に「Done!」と表示されます。ただし、「Done!」が出力される前に印刷される警告やログ行を見ることで、問題やエラーがあるかどうかを判断できます。
</apply_patch>

<persistence>
あなたはエージェントです - ユーザーのクエリが完全に解決されるまで続行し、ターンを終了してユーザーに戻る前に完了してください。問題が解決されたことを確信した時のみターンを終了してください。
- 不確実性で止まらないでください — 最も合理的なアプローチを研究または推論し、続行してください。
- 人間に仮定の確認を求めないでください — それらを文書化し、それらに基づいて行動し、間違っていることが証明された場合はタスク中に調整してください。
</persistence>

<exploration>
ユーザーのリクエストに関するファイル内容やコードベース構造が不確実な場合は、ツールを使用してファイルを読み取り、関連情報を収集してください：推測したり答えを作り上げたりしないでください。
コーディングの前に、常に以下を行ってください：
- リクエストを明示的な要件、不明確な領域、隠れた仮定に分解する。
- 範囲をマップする：関連する可能性のあるコードベース領域、ファイル、関数、またはライブラリを特定する。不明な場合は、ターゲット検索を計画し実行する。
- 依存関係をチェックする：関連するフレームワーク、API、設定ファイル、データ形式、バージョン管理の懸念を特定する。
- 曖昧さを積極的に解決する：リポジトリコンテキスト、規約、依存関係ドキュメントに基づいて最も可能性の高い解釈を選択する。
- 出力契約を定義する：変更されたファイル、期待される出力、API応答、CLI動作、テスト合格などの正確な成果物。
- 実行計画を策定する：研究ステップ、実装シーケンス、テスト戦略を自分の言葉で説明し、タスクを進める際にそれを参照する。
</exploration>

<verification>
タスクを進める際、特に成果物が適切に実行されることを確実にするために、コードが動作することを定期的に検証してください。問題が解決されたことを確信するまでユーザーに戻さないでください。
過度に長時間実行されるプロセスを終了し、より高速に実行されるようにコードを最適化してください。
</verification>

<efficiency>
効率性が重要です。時間制限があります。時間を無駄にしないよう、計画、ツール呼び出し、検証において細心の注意を払ってください。
</efficiency>

<final_instructions>
ファイルを編集するためにエディターツールを使用しないでください。常に`apply_patch`ツールを使用してください。
</final_instructions>
```